In [16]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

def get_dtlmenu_links(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    links = soup.find_all('a', {'class': 'hyakumeiten-shop__target js-sitecatalyst-link-track'})
    dtlmenu_urls = [link.get('href') if 'dtlmenu' in link.get('href') else link.get('href').rstrip('/') + '/dtlmenu/' for link in links]
    return dtlmenu_urls

def get_menu_data(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    # class="rstdtl-menu-lst__contents"で囲まれたデータから価格、画像URLを取得
    contents = soup.find_all('div', {'class': 'rstdtl-menu-lst__contents'})
    data = []
    for content in contents:
        price_p = content.find('p', {'class': 'rstdtl-menu-lst__price'})
        img_div = content.find('div', {'class': 'rstdtl-menu-lst__img'})
        
        # 価格と画像が存在する場合のみデータを取得
        if price_p is not None and img_div is not None:
            price = price_p.text
            img_url = img_div.find('a')['href']
            data.append((price, img_url))
    
    return data

# 入手した各urlからデータを取得
base_url = 'https://award.tabelog.com/hyakumeiten/teishoku'
dtlmenu_links = get_dtlmenu_links(base_url)
all_data = []
for link in dtlmenu_links:
    data = get_menu_data(link)
    all_data.extend(data)

# 出力
df = pd.DataFrame(all_data, columns=['price', 'img_url'])
df.to_csv('menu_data.csv', index=False)
